<br>

# <font color="#76b900">**Notebook 3:** Running State Chains</font>

<br>

In [ ]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio

## If you're in colab and encounter a typing-extensions issue,
##  restart your runtime and try again
# from langchain_nvidia_ai_endpoints._common import NVEModel
from getpass import getpass
import requests
import os

hard_reset = False  ## <-- Set to True if you want to reset your NVIDIA_API_KEY
while "nvapi-" not in os.environ.get("NVIDIA_API_KEY", "") or hard_reset:
    try: 
        response = requests.get("http://docker_router:8070/get_key").json()
        assert response.get('nvapi_key')
    except: response = {'nvapi_key' : getpass("NVIDIA API Key: ")}
    os.environ["NVIDIA_API_KEY"] = response.get("nvapi_key")
    try: requests.post("http://docker_router:8070/set_key/", json={'nvapi_key' : os.environ["NVIDIA_API_KEY"]}).json()
    except: pass
    hard_reset = False
    if "nvapi-" not in os.environ.get("NVIDIA_API_KEY", ""):
        print("[!] API key assignment failed. Make sure it starts with `nvapi-` as generated from the model pages.")

print(f"Retrieved NVIDIA_API_KEY beginning with \"{os.environ.get('NVIDIA_API_KEY')[:9]}...\"")
from langchain_nvidia_ai_endpoints._common import NVEModel
NVEModel().available_models

In [ ]:
import json
with open('nvidia_api_key.json', 'r') as f:
    json_data = json.load(f)

# Accede a la clave "api" en los datos cargados
nvidia_api_key = json_data["api"]

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from copy import deepcopy

inst_llm = ChatNVIDIA(model="mixtral_8x7b")  ## Feel free to change the models

prompt1 = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])
prompt2 =  ChatPromptTemplate.from_messages([
    ("system", (
        "Only responding in rhyme, change the topic of the input poem to be about {topic}!"
        " Make it happy! Try to keep the same sentence structure, but make sure it's easy to recite!"
        " Try not to rhyme a word with itself."
    )),
    ("user", "{input}")
])

## These are the main chains, constructed here as modules of functionality.
chain1 = prompt1 | inst_llm | StrOutputParser()  ## only expects input
chain2 = prompt2 | inst_llm | StrOutputParser()  ## expects both input and topic

################################################################################
## SUMMARY OF TASK: chain1 currently gets invoked for the first input.
##  Please invoke chain2 for subsequent invocations.

def rhyme_chat2_stream(message, history, return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
        if entry[0] and entry[1]:
            ## If a generation occurred as a direct result of a user input,
            ##  keep that response (the first poem generated) and break out
            first_poem = entry[1]
            break

    if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

        ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage

    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        buffer = f"Sure! Here you go!\n\n"
        yield buffer

        chat_gen = chain2.stream({"input" : first_poem, "topic" : message})
        for token in chat_gen:
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage

################################################################################
## Below: This is a small-scale simulation of the gradio routine.

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=5):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
history = [[None, "Let me help you make a poem! What would you like for me to write?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = rhyme_chat2_stream,
    history = history
)

In [ ]:
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,       ## Wrap an implicit "dictionary" runnable
    RunnablePassthrough,
)
from langchain.schema.runnable.passthrough import RunnableAssign

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, ChatMessage, AIMessage
from typing import Iterable
import gradio as gr

external_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a SkyFlow chatbot, and you are helping a customer with their issue. Please chat with them! Stay concise and clear!"
        " Your running knowledge base is: {know_base}. This is for you only; Do not mention it! \nUsing that, we retrieved the following: {context}\n"
        " If they provide info and the retrieval fails, ask to confirm their first/last name and confirmation. Do not ask them any other personal info."
        " If it's not important to know about their flight, do not ask. The checking happens automatically; you cannot check manually."
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

##########################################################################
## Knowledge Base Things

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: Optional[int] = Field(None, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: str = Field("", description="Topics that have not been resolved yet")
    current_goals: str = Field("", description="Current goal for the agent to address")

parser_prompt = ChatPromptTemplate.from_messages([
    ('system', (
        "You are a chat assistant representing the airline SkyFlow, and are trying to track info about the conversation. "
        "You have just recieved a message from the user. Please fill in the schema based on the chat. \n{format_instructions}"
    )), ('user', "{know_base} is the current state. New exchange: You said {output} and they replied {input}")
])

fail_str = (
    "You cannot access user's flight/account details until they provide "
    "first name, last name, and flight confirmation code."
)

## Your goal is to invoke the following through natural conversation
# get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
#     "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."

## TODO: Pick a working model that you're happy with
instruct_llm = (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())
chat_model = (ChatNVIDIA(model="mixtral_8x7b") | StrOutputParser())

## The external chain is already supported, and just streams the output
external_chain = external_prompt | chat_model

## The internal chain is invoked in the loop, but is not defined yet

#####################################################################################
## START TODO: Define the extractor and internal chain to satisfy the objective

extractor = RunnableAssign({'know_base' : RExtract(KnowledgeBase, instruct_llm, parser_prompt)})
internal_chain = extractor | RunnableAssign(
    {'context' : lambda d: itemgetter('know_base') | get_key | get_flight_info}
)

## END TODO
#####################################################################################

state = {'know_base' : KnowledgeBase()}

def chat_gen(message, history=[], return_buffer=False):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)
    print("State after chain run:", state)

    ## Streaming the results
    buffer = ""
    for token in external_chain.stream(state):
        buffer += token
        yield buffer

chatbot = gr.Chatbot(value = [[None, "Hello! I'm your SkyFlow agent! How can I help you?"]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    ## NOTE: This should also give you a temporary public link which can be
    ## used to access this info on the public web while the session is live.
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e